In [36]:
import numpy as np
import datetime

### Initialize ground truth

In [37]:
from stonesoup.types.array import StateVector, CovarianceMatrix
from stonesoup.types.state import GaussianState
initial_state_mean = StateVector([[0], [0], [0], [0]])
initial_state_covariance = CovarianceMatrix(np.diag([4,0.5,4,0.5]))
timestep_size = datetime.timedelta(seconds=5)
number_of_steps = 20
birth_rate = 0.3
death_probability = 0.05
start_time = datetime.datetime.now().replace(microsecond=0)
initial_state = GaussianState(initial_state_mean, initial_state_covariance, start_time)

### Create transition model


In [38]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(0.05), ConstantVelocity(0.05)])

### Create multi target ground truth simulator

In [39]:
from stonesoup.simulator.simple import MultiTargetGroundTruthSimulator
groundtruth_sim = MultiTargetGroundTruthSimulator(
    transition_model = transition_model,
    initial_state = initial_state,
    timestep = timestep_size,
    number_steps = number_of_steps,
    birth_rate = birth_rate,
    death_probability = death_probability
)

### Create measurement model

In [40]:
from stonesoup.simulator.simple import SimpleDetectionSimulator
from stonesoup.models.measurement.linear import LinearGaussian

measurement_model_covariance = np.diag([0.25, 0.25])
measurement_model = LinearGaussian(4, [0,2], measurement_model_covariance)

probability_detection = 0.9

clutter_area = np.array([[-1, 1], [-1, 1]])*100
clutter_rate = 1

In [41]:
detection_sim = SimpleDetectionSimulator(
    groundtruth = groundtruth_sim,
    measurement_model = measurement_model,
    detection_probability = probability_detection,
    meas_range = clutter_area,
    clutter_rate = clutter_rate

)

### Create predictor and updater

In [42]:
from stonesoup.predictor.kalman import KalmanPredictor
predictor = KalmanPredictor(transition_model)
from stonesoup.updater.kalman import KalmanUpdater
updater = KalmanUpdater(measurement_model)

### Create data associator (GNN)

In [43]:
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis
hypothesiser = DistanceHypothesiser(predictor, updater, measure = Mahalanobis(), missed_distance = 3)

In [44]:
from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
data_associator = GNNWith2DAssignment(hypothesiser)

### Create initiator and deleter

In [45]:
from stonesoup.deleter.error import CovarianceBasedDeleter
covariance_limit_for_delete = 2
deleter = CovarianceBasedDeleter(covar_trace_thresh = covariance_limit_for_delete)

s_prior_state = GaussianState([[0],[0],[0],[0]], np.diag([0, 0.5, 0, 0.5]))
min_detections = 3

from stonesoup.initiator.simple import MultiMeasurementInitiator
initiator = MultiMeasurementInitiator(
    prior_state = s_prior_state,
    measurement_model = measurement_model,
    deleter = deleter,
    data_associator = data_associator,
    updater = updater,
    min_points = min_detections
)

### Run the tracker

In [46]:
from stonesoup.tracker.simple import MultiTargetTracker

tracker = MultiTargetTracker(
    initiator = initiator,
    deleter = deleter,
    detector = detection_sim,
    data_associator = data_associator,
    updater = updater
)

In [47]:
groundtruth = set()
detections = set()
tracks = set()

for time, ctracks in tracker:
    groundtruth.update(groundtruth_sim.groundtruth_paths)
    detections.update(detection_sim.detections)
    tracks.update(ctracks)
    

In [48]:
from stonesoup.plotter import AnimatedPlotterly

timesteps = [start_time + timestep_size*k for k in range(number_of_steps)]

plotter = AnimatedPlotterly(timesteps, tail_length=1)
plotter.plot_ground_truths(groundtruth, mapping = [0,2])
plotter.plot_measurements(detections, mapping = [0,2])
plotter.plot_tracks(tracks, mapping = [0,2])
plotter.fig